In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import pyPyrTools as ppt
import math
from scipy import stats
from scipy import optimize as opt

In [ ]:
def transform_angle(x):
    """transform angle"""
    ang = x.angle
    if x.hemi == 'rh':
        ang = -ang
    ang = np.mod(np.radians(ang - 90), 2*np.pi)
    return ang

In [ ]:
df = pd.read_csv("/home/billbrod/Data/spatial_frequency_preferences/derivatives/first_level_analysis/stim_class/posterior/sub-wlsubj045/ses-02/sub-wlsubj045_ses-02_task-sfp_v1-2-3_e1-12_no_bin_summary.csv")
df['transformed_angle'] = df.apply(transform_angle, 1)

In [ ]:
df[(df.varea==1)&(df.R2>55)].drop_duplicates('voxel').sort_values('R2', ascending=False).head(5)[['voxel', 'R2', 'varea', 'hemi', 'angle' ,'eccen', 'transformed_angle']]

In [ ]:
# Pick a V1 voxel with a good R2
voxel_df = df[(df.voxel==9643)]
voxel_df.head(1)[['voxel', 'varea', 'R2', 'eccen', 'transformed_angle', 'hemi']]

In [ ]:
voxel_eccen = voxel_df.eccen.unique()[0]
voxel_angle = voxel_df['transformed_angle'].unique()[0]
resp_df = []
for i, row in voxel_df.iterrows():
    dx, dy, mag, _ = sfp.stimuli.sf_cpd(1080, 24, voxel_eccen, voxel_angle, w_r=row.w_r, w_a=row.w_a)
    dr, da = sfp.stimuli.sf_origin_polar_cpd(1080, 24, voxel_eccen, voxel_angle, w_r=row.w_r, w_a=row.w_a)
    resp_df.append(pd.DataFrame({'w_x':dx, 'w_y':dy, 'response':row.amplitude_estimate_median, 'stim_class':row.stimulus_superclass, 'local_wr':dr, 'local_wa':da,
                                 'global_wr':row.w_r, 'global_wa':row.w_a, 'eccen': voxel_eccen, 'angle': voxel_angle, 'mag': mag}, [i]))
resp_df = pd.concat(resp_df)

resp_df['xy_distance'] = np.sqrt(resp_df.w_x**2 + resp_df.w_y**2)
resp_df['xy_angle'] = np.arctan2(resp_df.w_y, resp_df.w_x)
resp_df['ra_distance'] = np.sqrt(resp_df.local_wr**2 + resp_df.local_wa**2)
resp_df['ra_angle'] = np.arctan2(resp_df.local_wa, resp_df.local_wr)

In [ ]:
resp_df.head()

In [ ]:
sizes = resp_df['response'].values
sizes = (sizes - sizes.min()) / (sizes.max() - sizes.min())
g=sns.FacetGrid(resp_df, hue='stim_class', size=5, aspect=1)
g.map(plt.scatter, 'local_wr', 'local_wa', s=sizes*80)
g.add_legend()
scatter_ax = plt.gca()
scatter_ax.set_aspect('equal')

In [ ]:
sizes = resp_df['response'].values
sizes = (sizes - sizes.min()) / (sizes.max() - sizes.min())
g=sns.FacetGrid(resp_df, hue='stim_class', size=5, aspect=1)
g.map(plt.scatter, 'local_wr', 'local_wa', s=sizes*80)
g.add_legend()
scatter_ax = plt.gca()
scatter_ax.set_xscale('symlog', basex=2, linthreshx=2**(-3))
scatter_ax.set_yscale('symlog', basey=2, linthreshy=2**(-3))

In [ ]:
def twoD_Gaussian((x, y), amplitude, mu_x, mu_y, sig_xx, sig_xy, sig_yx, sig_yy):
    twod_gauss = stats.multivariate_normal(mean=(mu_x, mu_y), cov=[[sig_xx, sig_xy],[sig_yx, sig_yy]])
    xgrid, ygrid = np.meshgrid(x, y)
    X = np.empty(xgrid.shape + (2,))
    X[:,:,0] = xgrid
    X[:,:,1] = ygrid
    gauss_resp = twod_gauss.pdf(X)
    gauss_resp /= gauss_resp.max()
    return amplitude * gauss_resp
def fit_twoD_Gaussian(*args, **kwargs):
    return np.diagonal(twoD_Gaussian(*args, **kwargs))

def loggaussian_donut((x, y), amplitude, major_axis, minor_axis, major_axis_sigma, minor_axis_sigma, rotation_angle):

    xgrid, ygrid = np.meshgrid(x, y)
    x_rotated = xgrid*np.cos(rotation_angle) - ygrid*np.sin(rotation_angle) 
    y_rotated = xgrid*np.sin(rotation_angle) + ygrid*np.cos(rotation_angle)
    r = np.log(np.sqrt(x_rotated**2 + y_rotated**2))
    th = np.arctan2(y_rotated, x_rotated)

    # transform angles based on ellipse axes
    transformed_theta = np.arctan2(major_axis*np.sin(th), minor_axis*np.cos(th))

    # Gaussian center as function of angle
    ctr = np.sqrt((major_axis*np.cos(transformed_theta))**2 + (minor_axis*np.sin(transformed_theta))**2)

    # rotational sigma
    sigma = np.sqrt((major_axis_sigma*np.cos(transformed_theta))**2 + (minor_axis_sigma*np.sin(transformed_theta))**2)
    
    # This is our function
    return amplitude*np.exp(-(r-ctr)**2 / (2*sigma**2))

def fit_loggaussian_donut(*args, **kwargs):
    return np.diagonal(loggaussian_donut(*args, **kwargs))

x=np.linspace(-8, 8, 101)
donut = loggaussian_donut((x,x), 2, 1.5, .5, .2, .2, np.pi/2.)
plt.imshow(donut, extent=(x.min(), x.max(), x.min(), x.max()), cmap='gray')
plt.colorbar()

In [ ]:
x= resp_df['w_x']
y= resp_df['w_y']
# popt, pcov = opt.curve_fit(fit_twoD_Gaussian, (x, y), resp_df['response'], p0=(20, 0, 0, 1, 0, 0, 1))
popt, pcov = opt.curve_fit(fit_loggaussian_donut, (x, y), resp_df['response'], p0=(20, 1, 1, .2, .2, 0), bounds=(0, [np.inf, np.inf, np.inf, np.inf, np.inf, np.pi]))
x = np.linspace(-12, 6, 1000)
y = np.linspace(-6, 12, 1000)
donut = loggaussian_donut((x,y), *popt)
plt.contourf(x,y,donut, aspect='equal', cmap="RdBu", norm=sfp.plotting.MidpointNormalize(midpoint=0))
ax = plt.gca()
#ax.set_xlim(scatter_ax.get_xlim())
#ax.set_ylim(scatter_ax.get_ylim())
plt.colorbar()
popt

In [ ]:
x= resp_df['local_wr']
y= resp_df['local_wa']
# popt, pcov = opt.curve_fit(fit_twoD_Gaussian, (x, y), resp_df['response'], p0=(20, 0, 0, 1, 0, 0, 1))
popt, pcov = opt.curve_fit(fit_loggaussian_donut, (x, y), resp_df['response'], p0=(20, 1, 1, .2, .2, 0), bounds=(0, [np.inf, np.inf, np.inf, np.inf, np.inf, np.pi]))
x = np.linspace(-12, 6, 1000)
y = np.linspace(-6, 12, 1000)
donut = loggaussian_donut((x,y), *popt)
plt.contourf(x,y,donut, aspect='equal', cmap="RdBu", norm=sfp.plotting.MidpointNormalize(midpoint=0))
ax = plt.gca()
#ax.set_xlim(scatter_ax.get_xlim())
#ax.set_ylim(scatter_ax.get_ylim())
plt.colorbar()
popt

In [ ]:
plt.figure(figsize=(8,5))
plt.scatter(resp_df['local_wr'], resp_df['local_wa'], c=resp_df['response'], s=sizes*80,cmap='Blues', vmin=0, vmax=resp_df['response'].max())
plt.colorbar()
plt.contour(x, y, loggaussian_donut((x, y), *popt), aspect='equal', cmap='Blues', vmin=0, vmax=resp_df['response'].max())# levels=[0, 2.5, 5, 7.5, 10, 12.5, 15, 17.5, 19.9, 20.5])
plt.colorbar()
ax = plt.gca()
ax.set_xlim(scatter_ax.get_xlim())
ax.set_ylim(scatter_ax.get_ylim())


In [ ]:
g = sns.PairGrid(resp_df, x_vars=['w_x', 'w_y', 'xy_distance', 'xy_angle'], y_vars=['response'], hue='xy_angle')
g.map(plt.scatter)
g.add_legend()

In [ ]:
g = sns.PairGrid(resp_df, x_vars=['local_wa', 'local_wr', 'ra_distance', 'ra_angle'], y_vars=['response'], hue='ra_angle')
g.map(plt.scatter)
g.add_legend()